<a href="https://colab.research.google.com/github/retico/cmepda_medphys/blob/master/L1_code/Lecture1_demo2_read_DICOM_dir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline

# Reading the dataset from Google Drive
Prior to this operation be sure to have added the shared folder to your Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls "/content/drive/My Drive/"

In [ ]:
!ls "/content/drive/My Drive/cmepda_medphys_dataset/IMAGES/DICOM_Examples"

In [ ]:
!cp "/content/drive/My Drive/cmepda_medphys_dataset/IMAGES/DICOM_Examples/Lung_CT_cd2-20191122T084339Z-001.zip" /content/

In [ ]:
!ls /content/

In [ ]:
!unzip /content/Lung_CT_cd2-20191122T084339Z-001.zip

In [ ]:
!ls /content/Lung_CT_cd2/

In [ ]:
DATASETS = "/content/Lung_CT_cd2"

## Demo 2: reading a DICOM directory using the pydicom package

This example illustrates how to use the pydicom package to open a DICOM directory, 
print some dataset information (metadata), and view the images on different planes using matplotlib.


In [ ]:
!pip install pydicom

In [ ]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
import pydicom
from pydicom.filereader import read_dicomdir

We use the `glob.glob` to get a list of all the files matching the pattern *data_path/\* *, i.e. data_path/whatever

In [ ]:
data_path = os.path.join(DATASETS, "12031814")
dicom_files = glob.glob(os.path.join(data_path, '*'))

In [ ]:
# Print out the first 5 file names to verify we're in the right folder.
print("Total of {} DICOM images.\nFirst 5 filenames:".format(len(dicom_files)))
dicom_files[:5]

Let's try to open the first one as a check

In [ ]:
dataset = pydicom.dcmread(dicom_files[0])

plt.imshow(dataset.pixel_array, cmap=plt.cm.bone)
plt.show()


# Load dicom files into Numpy arrays

In [ ]:
import numpy as np
import pydicom
import glob
import matplotlib.pyplot as plt

In [ ]:
data_path = os.path.join(DATASETS, "12031814")
dicom_files = glob.glob(os.path.join(data_path, '*'))

From the first dicom file, I read some useful data such as the number of rows and columns, and how the image intensities are represented, i.e. int, uint16, float, etc.

In [ ]:
dataset = pydicom.read_file(dicom_files[0])

In [ ]:
shape = dataset.Rows, dataset.Columns, len(dicom_files)
shape

In [ ]:
dataset.pixel_array.dtype

For the sake of efficiency, we prealloacate the memory required to store the images

In [ ]:
CT_array = np.zeros(shape, dtype=dataset.pixel_array.dtype)
for i, fname in enumerate(dicom_files):
    CT_array[:,:,i] = pydicom.read_file(fname).pixel_array

In [ ]:
CT_array.shape

In [ ]:
ct = CT_array[:,:, CT_array.shape[2]//2 ]
plt.imshow(ct, cmap='gray')

In [ ]:
ct = CT_array[:, CT_array.shape[1]//2 , :]
plt.imshow(ct, cmap='gray')

## A more correct approach

We assumed that dicom files are lexicographically ordered but to be sure to order the columns correctly we must read some information about the images from the dicom header.

In [ ]:
import numpy as np
import pydicom
import glob
import os

In [ ]:
data_path = os.path.join(DATASETS, "12031814")
dicom_files = glob.glob(os.path.join(data_path, '*'))

**Slice Location (0020,1041)** is defined as the relative position of the image plane expressed in mm.
This information is relative to an unspecified implementation specific reference point.

After reading all of the dicom files, we filter out the ones without the SliceLocation field

In [ ]:
dicom_slices = [ pydicom.read_file(fname) for fname in dicom_files]
slices = [dcm_slice for dcm_slice in dicom_slices if hasattr(dcm_slice, 'SliceLocation')]

In [ ]:
slices[0].SliceLocation

In [ ]:
plt.imshow(slices[0].pixel_array, cmap=plt.cm.bone)
plt.show()

In [ ]:
len(slices)

Let's sort the list by means of the slice location attribute, i.e. the correct order

In [ ]:
slices.sort(key=lambda x: x.SliceLocation)

We now build a NumPy array which contains the images

In [ ]:
shape = slices[0].Rows, slices[0].Columns, len(slices)

CT_array = np.zeros(shape, dtype=slices[0].pixel_array.dtype)
for i, dcm in enumerate(slices):
    CT_array[:,:,i] = dcm.pixel_array

In [ ]:
CT_array.shape

In [ ]:
plt.imshow(CT_array[:, CT_array.shape[1]//2,:], cmap='gray')

When plotting the different views, it is necessary to preserve the aspect ratio 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
x, y, z = *slices[0].PixelSpacing, slices[0].SliceThickness
x,y,z

In [ ]:
aspect_ratio = {
    'axial': y/x,
    'sagittal': y/z,
    'coronal': x/z
}

In [ ]:
# plot 3 orthogonal slices
a1 = plt.subplot(2, 2, 3)
plt.imshow(CT_array[:, :, CT_array.shape[2]//2], cmap='gray')
a1.set_aspect(aspect_ratio['axial'])
a1.axis('off')
a1.set_title('Axial')

a2 = plt.subplot(2, 2, 1)
ct = CT_array[:, CT_array.shape[1]//2 , :].T
ct = np.flipud(ct)
plt.imshow(ct, cmap='gray')
a2.set_aspect(aspect_ratio['sagittal'])
a2.axis('off')
a2.set_title('Sagittal')


a3 = plt.subplot(2, 2, 2)

ct = CT_array[CT_array.shape[0]//2, :, :].T
ct = np.flipud(ct)

plt.imshow(ct, cmap='gray')
a3.set_aspect(aspect_ratio['coronal'])
a3.axis('off')
a3.set_title('Coronal');


# Another mistake: different series

Actually, we made a mistake. In the folder, the files belong to different series. Let's find out how many!

In [ ]:
from collections import Counter

In [ ]:
count_series = Counter([slice_.SeriesNumber for slice_ in slices])
count_series

Let's build the series "3" list

In [ ]:
slices_3 = [slice_ for slice_ in slices if slice_.SeriesNumber == 3]
len(slices_3)

In [ ]:
slices_3[0].SeriesDescription

Using a dictionary we can build the lists for all the different  series

In [ ]:
series = {k: [s for s in slices if s.SeriesNumber == k] for k in count_series.keys()}
series.keys()

In [ ]:
len(series[7])